# The book-reading service research

Our company decided to be on the wave and bought a large service for reading books by subscription. Our first job as an analyst is to analyze the database.
It contains information about books, publishers, authors, as well as user reviews of books. This data will help formulate a value proposition for a new product.

## Importing Pandas and SQL

In [1]:
import pandas as pd

In [2]:
from sqlalchemy import create_engine

db_config = {'user': 'praktikum_student', 
'pwd': 'Sdf4$2;d-d30pp', 
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, 
'db': 'data-analyst-final-project-db'} 
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

*We will check the work with simple queries, display the first rows of all tables*

In [3]:
query = '''
SELECT *
FROM Books
LIMIT 5;
''' 

In [ ]:
pd.io.sql.read_sql(query, con = engine) 

*With a separate query, we display the total number of table rows*


In [ ]:
query = '''SELECT COUNT(1) FROM Books;''' 

In [ ]:
pd.io.sql.read_sql(query, con = engine) 

In [ ]:
query = '''
SELECT *
FROM authors
LIMIT 5
''' 

In [ ]:
pd.io.sql.read_sql(query, con = engine)

In [ ]:
query = '''SELECT COUNT(1) FROM authors;''' 

In [ ]:
pd.io.sql.read_sql(query, con = engine) 

In [ ]:
query = '''
SELECT *
FROM publishers
LIMIT 5
''' 

In [ ]:
pd.io.sql.read_sql(query, con = engine)

In [ ]:
query = '''SELECT COUNT(1) FROM publishers;''' 

In [ ]:
pd.io.sql.read_sql(query, con = engine) 

In [ ]:

query = '''
SELECT *
FROM reviews
LIMIT 5
''' 

In [ ]:
pd.io.sql.read_sql(query, con = engine)

In [ ]:
query = '''SELECT COUNT(1) FROM reviews;''' 

In [ ]:
pd.io.sql.read_sql(query, con = engine) 

In [ ]:

query = '''
SELECT *
FROM ratings
LIMIT 5
''' 

In [ ]:
pd.io.sql.read_sql(query, con = engine)

In [ ]:
query = '''SELECT COUNT(1) FROM ratings;''' 

In [ ]:
pd.io.sql.read_sql(query, con = engine) 

<p align="center"><img src='https://concrete-web-bad.notion.site/image/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fsecure.notion-static.com%2F069818d1-0e5c-4d87-a461-0de584ab9c33%2FUntitled_(33).png?table=block&id=bd53c8db-b4fd-49eb-8cc9-572ebb3c9163&spaceId=9e4bd47b-c6e6-4ca3-bcee-279794b47315&width=1640&userId=&cache=v2'></p>


### Description of data

**Table `books`**

Contains information about books:

- `book_id` — book identifier;
- `author_id` — author identifier;
- `title` — title of the book;
- `num_pages` - number of pages;
- `publication_date` - publication date of the book;
- `publisher_id` - publisher identifier.

**The `authors` table**

Contains information about the authors:

- `author_id` — author identifier;
- `author` is the name of the author.

**Table `publishers`**

Contains information about publishers:

- `publisher_id` — publisher identifier;
- `publisher` — publisher name;

**Table `ratings`**

Contains data about user ratings of books:

- `rating_id` — rating identifier;
- `book_id` — book identifier;
- `username` - the name of the user who left the rating;
- `rating` - rating of the book.

**Table `reviews`**

Contains data about user reviews of books:

- `review_id` — review identifier;
- `book_id` — book identifier;
- `username` - the name of the user who wrote the review;
- `text` — review text.

*Let's look at other tables in the database and what types of data are in the tables of interest to us*

In [ ]:

display(pd.io.sql.read_sql('''

SELECT *
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog' AND
      schemaname != 'information_schema';

''', con = engine))


display(pd.io.sql.read_sql('''
SELECT 
    table_name, 
    column_name, 
    data_type, 
    is_nullable
FROM INFORMATION_SCHEMA.COLUMNS 
WHERE table_name IN ('books', 'authors', 'publishers', 'ratings', 'reviews');
''', con = engine))

## Research

### Let's count how many books came out after January 1, 2000;


In [ ]:
query = '''
SELECT COUNT (*)
FROM books
WHERE publication_date >=  '2000-01-01'
''' 

In [ ]:
pd.io.sql.read_sql(query, con = engine)

*Since January 1, 2000, 821 books have been published*

### For each book, we will calculate the number of reviews and the average rating;

In [ ]:
query = '''
WITH rat AS 
(SELECT book_id,
ROUND (AVG (rating), 3) AS average_rating
FROM ratings
GROUP BY book_id),
rew AS 
(SELECT book_id,
COUNT (DISTINCT review_id) AS review_number
FROM reviews
GROUP BY book_id)

SELECT b.title,
       a.author,
        rat.average_rating,
        rew.review_number
       
FROM books as b 
LEFT JOIN rat ON b.book_id = rat.book_id
LEFT JOIN rew ON b.book_id = rew.book_id
LEFT JOIN authors a ON  a.author_id = b.author_id
ORDER BY average_rating DESC, review_number DESC
Limit 10
''' 

In [ ]:
pd.io.sql.read_sql(query, con = engine)

*Мы получили топ-10 Самых популярных книг в нашем сервисе*

### Let's determine the publisher has released the largest number of books

We will take only books thicker than 50 pages - exclude brochures from the analysis

In [ ]:
query = '''
WITH big AS
(SELECT book_id,
publisher_id
FROM books
WHERE num_pages>50)

SELECT p.publisher,
COUNT (DISTINCT big.book_id) AS book_number
FROM publishers p
LEFT JOIN big ON p.publisher_id = big.publisher_id
GROUP BY p.publisher
ORDER BY book_number DESC
Limit 10
''' 

In [ ]:
pd.io.sql.read_sql(query, con = engine)

*Received the most prescriptive publishing houses*

### Let's determine the author with the highest average book rating

*We will take only books with 50 grades or more*

In [ ]:
query = '''
SELECT a.author,
AVG (r.rating) AS average_rating
FROM authors a
LEFT JOIN books b ON a.author_id = b.author_id
LEFT JOIN ratings r ON b.book_id = r.book_id
WHERE b.book_id in  (SELECT b.book_id
                   FROM books b 
                   LEFT JOIN ratings r ON b.book_id = r.book_id 
                   GROUP BY b.book_id 
                   HAVING COUNT (DISTINCT r.rating_id) >=50)

GROUP BY a.author
ORDER BY average_rating DESC
LIMIT 1
''' 

In [ ]:
pd.io.sql.read_sql(query, con = engine)

*Thus, the highest-rated author is Harry Potter author J.K. Rowling with illustrations by Mary GrandPré (average rating of their books 4.29)*

### Let's calculate the average number of reviews from users

We will consider only users who have put in more than 50 grades.

In [ ]:
query = '''
SELECT AVG (group_rev.review_number)
FROM (SELECT username,
      COUNT (DISTINCT review_id) AS review_number
      FROM reviews
      WHERE username in (SELECT username
                         FROM ratings
                         GROUP BY username
                         HAVING COUNT (DISTINCT rating_id) >=50)
      GROUP BY username) AS group_rev

''' 

In [ ]:
pd.io.sql.read_sql(query, con = engine)

*The active service users average write 24 reviews*

## **CONCLUSIONS**


1. We examined the database of the book reading service.
2. Got the most popular books to promote on promo.
3. We found the most productive publishing houses for the convenience of buying rights to books.
4. The most beloved Author was determined - JK Rowling.
5. We found out how many reviews are written by active users on average - 24
6. Find out how many books in our library have been published since January 1, 2001